# Build point array given

In [1]:
! pip install haversine
! pip install vincenty

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [2]:
from haversine import *
import pandas as pd
from tqdm import tqdm
from helpers import *

In [3]:
# Bounding box over london
bottom_left = (51.251246, -0.621715)
top_right = (51.715616, 0.369802)

top_left = (51.715616, -0.621715)
bottom_right = (51.251246, 0.369802)

Algorithm Key points:
- Points must be evenly spaced creating circles with a chosen diameter.
- Number of points spaced along the vertical will be consistent anywhere you move horizontally
- However, number points spaced along the horizonal will chance as you move vertically

1. Choose diameter of circle
2. Divide vertical length by diameter to work out number of rows
3. Now iterate through rows 
for i in range(len(rows)):
    - i. Get row start point and row end point moving right to left by moving r*diameter down from top left and top right
    - ii. calcualte horizontal distance to right line
    - iii. Divide by diameter of circle to get circles on the row
    - iv. If there's a remainder, divide by 2 and shift along to the right
    - v. create calcuated number of circles on the row


In [5]:
# 1. Choose diameter of circle in km
d = 1

# 2. Divide vertical length by diameter to work out number of rows
length = haversine(top_left, bottom_left)
num_rows = int(length // d)
print(num_rows)

# 3. iterate through rows
points = []
for r in tqdm(range(num_rows)):
  # i. Get row start point and row end point moving right to left by moving r*diameter down from top left and top right
  row_start_point = inverse_haversine(top_left, r * d, Direction.SOUTH)
  row_end_point = inverse_haversine(top_right, r * d, Direction.SOUTH)
  
  # ii. calcualte horizontal distance to right line
  row_width = haversine(row_start_point, row_end_point)
  
  #iii. Divide by diameter of circle to get circles on the row
  number_points_on_row = int(row_width // d)
  
  # iv. If there's a remainder, divide by 2 and shift along to the right
  shift = ((row_width / d) % 1) / 2
  row_start_point = inverse_haversine(row_start_point, shift, Direction.EAST)
  
  # v. create calcuated number of circles on the row
  points_in_row_list = []
  for i in range(number_points_on_row):
    points_in_row_list.append(inverse_haversine(row_start_point, i*d, Direction.EAST))
  
  points = points + points_in_row_list
  
print("done")

51


100%|██████████| 51/51 [00:00<00:00, 3716.67it/s]

done


## Using the function:

In [4]:
point_set_df = get_spaced_point_set_in_bbox(0.25, bottom_left, top_right)

point_set_df['Magnitude'] = pd.Series([1 for i in range(len(point_set_df))])
point_set_df

206


100%|██████████| 206/206 [00:00<00:00, 728.86it/s]


,Latitude,Longitude,Magnitude
0,51.715616,-0.620041,1
1,51.715616,-0.616412,1
2,51.715616,-0.612784,1
3,51.715615,-0.609155,1
4,51.715615,-0.605526,1
...,...,...,...
56458,51.250677,0.351799,1
56459,51.250648,0.355390,1
56460,51.250618,0.358982,1
56461,51.250587,0.362574,1


### Save in CSV for plotly

In [29]:
point_set_df.to_csv('./data/point_array_025.csv')

In [20]:
point_set_df = pd.read_csv('./data/point_array_01.csv', index_col=0)
point_set_df

,Latitude,Longitude,Magnitude
0,51.715616,-0.617249,1
1,51.715616,-0.609992,1
2,51.715615,-0.602734,1
3,51.715614,-0.595476,1
4,51.715612,-0.588219,1
...,...,...,...
14077,51.253131,0.333817,1
14078,51.253073,0.341000,1
14079,51.253014,0.348184,1
14080,51.252955,0.355368,1
